<a href="https://colab.research.google.com/github/Mohamed-Elnageeb/AI-algorithms/blob/main/CodeWeekend_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import userdata
api_token = 'mfzviboixcgjafzqzgblsrkuzdipdias'
api_url = 'https://codeweekend.dev:3721/api/'
files_url = 'https://codeweekend.dev:81/'

print('API TOKEN:', api_token)
print('API URL:', api_url)

import requests
import json
import time
import os

headers = {
    'authorization': f'Bearer {api_token}'
}

def show(inner_json):
    print(json.dumps(inner_json, indent=2))

def get_scoreboard():
    return requests.get(api_url + 'scoreboard', headers=headers).json()

def get_team_dashboard():
    return requests.get(api_url + 'team_dashboard', headers=headers).json()

def get_test(task_id):
    task_id_padded = '{:03d}'.format(task_id)
    url = f'{files_url}{task_id_padded}.json'
    return requests.get(url, headers=headers).content

# Returns at most 50 submissions
def get_team_submissions(offset=0, task_id=None):
    url = f'{api_url}team_submissions?offset={offset}'
    if task_id is not None:
      url += f'&task_id={task_id}'
    return requests.get(url, headers=headers).json()

def get_submission_info(submission_id, wait=False):
    url = f'{api_url}submission_info/{submission_id}'
    res = requests.get(url, headers=headers).json()
    if 'Pending' in res and wait:
        print('Submission is in Pending state, waiting...')
        time.sleep(1)
        return get_submission_info(submission_id)
    return res

# Returns submission_id
def submit(task_id, solution):
    res = requests.post(url = f'{api_url}submit/{task_id}',
                        headers=headers, files={'file': solution})
    if res.status_code == 200:
        return res.text
    print(f'Error: {res.text}')
    return None

def download_submission(submission_id):
    import urllib.request
    url = f'{api_url}download_submission/{submission_id}'
    opener = urllib.request.build_opener()
    opener.addheaders = headers.items()
    urllib.request.install_opener(opener)
    try:
        file, _ = urllib.request.urlretrieve(url, "downloaded.txt")
    except Exception as e:
        print('Failed to download submission: ', e)
        return None
    content = open(file, "r").read()
    os.remove(file)
    return content

def update_display_name(new_name):
  url = api_url + 'update_user'
  data = {
      'display_name': new_name,
      'email': "",
      'team_members': ""
  }
  return requests.post(url, json=data, headers=headers).content

# show(get_scoreboard())
# show(get_submission_info(427))
# show(get_team_dashboard())
# show(get_team_submissions())
# download_submission(476)
# get_test(1)
# update_display_name('Test 123')

API TOKEN: mfzviboixcgjafzqzgblsrkuzdipdias
API URL: https://codeweekend.dev:3721/api/


# Taking in input


In [23]:
from types import GeneratorType
import json
from bisect import bisect_right
from math import exp

input_file_name = r"027.json"

test_num = 27
inp = json.loads(get_test(test_num))
out = open("out.txt", "w")

num_turns = inp["num_turns"]
w, h = inp["width"], inp["height"]
hx, hy = inp["start_x"], inp["start_y"]

hero = inp["hero"]
base_speed = hero["base_speed"]
base_power = hero["base_power"]
base_range = hero["base_range"]
level_speed_coeff = hero["level_speed_coeff"]
level_power_coeff = hero["level_power_coeff"]
level_range_coeff = hero["level_range_coeff"]

monsters = list(map(lambda m: (m["x"], m["y"], m["hp"], m["exp"], m["gold"]), inp["monsters"]))

# DP Solution


In [24]:
def bootstrap(f):
    stack = []
    def wrappedfunc(*args, **kwargs):
        if stack:
            return f(*args, **kwargs)
        else:
            to = f(*args, **kwargs)
            while True:
                if type(to) is GeneratorType:
                    stack.append(to)
                    to = next(to)
                else:
                    stack.pop()
                    if not stack:
                        break
                    to = stack[-1].send(to)
            return to
    return wrappedfunc

In [25]:
# monsters = [(i,*monsters[i]) for i in range(len(monsters))]

# # monsters.sort(key= lambda x: ( ( hx-x[1] )**2 + (hy-x[2] )**2 ) )  #sort by euclidean distance
# monsters.sort(key= lambda x: ( x[3]/base_power + 2*(( hx-x[1] )**2 + (hy-x[2] )**2   )/base_speed - x[4] + x[5]  ) ) #sort by no of moves

# monsters = monsters[:7]
# levels = [1000,2100,3400,5000,7000]

# for l in range(6,30) :
#     levels.append(levels[-1] + 1000 + l*(l-1)*50)
# print(monsters)

# aft = {}
# pw = [1<<i for i in range(100)]

# bootstrap
# def f(i,x,y,mask,e = 0,fat=0) :
#     if i == num_turns :
#         yield 0

#     if dp[i][x][y][mask] != -1 :
#         yield dp[i][x][y][mask]

#     res = -1
#     rem = num_turns - i

#     L = bisect_right(levels,e)
#     power = int(base_power * (1 + L*level_power_coeff/100))

#     for nx,ny in posi_move[x,y]:
#         ans = (yield f(i+1,nx,ny,mask,e))
#         if res < ans :
#             aft[((i,x,y,mask))] =  (i+1,nx,ny,mask,("move",(nx,ny)))
#             res = ans

#     for b,mx,my,hp,exp,gold in posi_atk[x,y] :
#         need = ((hp + power - 1) // power)
#         if rem >= need and not mask&(b)  :

#             ans = (yield f(i+need,x,y,mask|(b),e+exp)) + gold
#             if res < ans :
#                 aft[((i,x,y,mask))] =  (i+need,x,y,mask|b,("attack",need,b))
#                 res = ans


#     dp[i][x][y][mask] = res
#     yield res



# def solve():
#     global posi_atk , posi_move

#     posi_atk = {}
#     posi_move = {}
#     max_w = max_h = 1
#     min_w = w
#     min_h = h

#     for i in range(1,w+1) :
#         for j in range(1,h+1) :
#             posi_atk[i,j] = []
#             ind = -1
#             for k,x,y,hp,exp,gold in monsters:
#                 ind += 1
#                 max_w = max(max_w,x)
#                 max_h = max(max_h,y)
#                 min_w = min(min_w,x)
#                 min_h = min(min_h,y)
#                 if (x-i)**2 + (y-j)**2 <= base_range**2 :
#                     try :
#                         posi_atk[i,j].append((1<<ind,x,y,hp,exp,gold))
#                     except :
#                         posi_atk[i,j] = [((1<<ind,x,y,hp,exp,gold))]

#     max_w = max(max_w,hx)
#     max_h = max(max_h,hy)
#     min_w = min(min_w,hx)
#     min_h = min(min_h,hy)
#     check = 0

#     for i in range(1,w+1) :
#         for j in range(1,h+1) :
#             if (not min_h <= j <= max_h) or (not min_w <= i <= max_w) :
#                 continue
#             posi_move[i,j] = []
#             for x in range(max(1,i-base_speed) , min(w,i+base_speed) + 1) :
#                 for y in range(max(1,j-base_speed) , min(h,j+base_speed) + 1) :
#                     if (x,y) == (i,j) or (not min_h <= y <= max_h) or (not min_w <= x <= max_w) :
#                         continue
#                     if (x-i)**2 + (y-j)**2 <= base_speed**2 and (x-i)**2 + (y-j)**2 >= (base_speed-1)**2 :
#                         try :
#                             posi_move[i,j].append((x,y))
#                         except :
#                             posi_move[i,j] = [((x,y))]

#             check =  max(check,len(posi_move[i,j]))
#     print(min_h,max_h,min_w,max_w,check)
#     print(f(0,hx,hy,0))



# dp = [[[[-1 for _ in range(2**len(monsters))] for _ in range(h+1) ] for _ in range(w+1)] for _ in range(num_turns)]

# solve()

In [26]:
# moves = []
# turn = 0
# cur_x = hx
# cur_y = hy
# cur_mask = 0
# while turn < num_turns :
#     turn,cur_x,cur_y,cur_mask , desc = aft[turn,cur_x,cur_y,cur_mask ]
#     moves.append(desc)
# # moves.reverse()
# print(moves)

# Simulated Annealing Solution


In [27]:
monsters = [(i,*monsters[i]) for i in range(len(monsters))]

# monsters.sort(key= lambda x: ( ( hx-x[1] )**2 + (hy-x[2] )**2 ) )  #sort by euclidean distance
monsters.sort(key= lambda x: ( 3*x[3]/base_power + 2*(( hx-x[1] )**2 + (hy-x[2] )**2   )/base_speed - 10*x[4] - 59*x[5]  ) ) #sort by no of moves
monsters.sort(key= lambda x: ( 3*x[3]/base_power + 2*(( hx-x[1] )**2 + (hy-x[2] )**2   )/base_speed - 6*x[4] - x[5]  ) ) #sort by no of moves
# random.shuffle(monsters)
# monsters = monsters[:15]

levels = [1000,2100,3400,5000,7000]

for l in range(6,300000) :
    levels.append(levels[-1] + 1000 + l*(l-1)*50)
# print(monsters)


import random
random.seed(10)

def objective_function(sol,ret) :
    cur_x = hx
    cur_y = hy
    cur_fat,cur_g,cur_exp = 0,0,0
    cur_range,cur_power , cur_speed = base_range,base_power,base_speed
    i = 0
    j = -1
    sol_moves = []

    while i < num_turns and j + 1 < len(sol) :
        j += 1
        ind,next_x,next_y,hp,exp,gold = sol[j]
        if j + 1 < len(sol) :
            ind2,next_x2,next_y2,_,_,_ = sol[j+1]
        else:
            ind2,next_x2,next_y2 = j+1,cur_x,cur_y

        while i < num_turns and (cur_x-next_x)**2 + (cur_y-next_y)**2 > cur_range**2 :
            i += 1
            bestval = float("inf")
            for nx,ny in posi_move[cur_x,cur_y] :
                if ((nx-next_x)**2 + (ny-next_y)**2 )  < bestval :
                    bestval = ((nx-next_x)**2 + (ny-next_y)**2 )
                    cur_x ,cur_y = nx,ny
                if ((nx+(cur_speed-base_speed)-next_x)**2 + (ny-next_y)**2)  < bestval :
                    bestval =  ((nx+(cur_speed-base_speed)-next_x)**2 + (ny-next_y)**2)
                    cur_x ,cur_y = nx+(cur_speed-base_speed),ny
                if ((nx-(cur_speed-base_speed)-next_x)**2 + (ny-next_y)**2) < bestval :
                    bestval = ((nx-(cur_speed-base_speed)-next_x)**2 + (ny-next_y)**2)
                    cur_x ,cur_y = nx-(cur_speed-base_speed),ny
                if ((nx-next_x)**2 + (ny+(cur_speed-base_speed)-next_y)**2 )  < bestval :
                    bestval = ((nx-next_x)**2 + (ny+(cur_speed-base_speed)-next_y)**2 )
                    cur_x ,cur_y = nx,ny+(cur_speed-base_speed)
                if ((nx-next_x)**2 + (ny-(cur_speed-base_speed)-next_y)**2)  < bestval :
                    bestval = 2*((nx-next_x)**2 + (ny-(cur_speed-base_speed)-next_y)**2)
                    cur_x ,cur_y = nx,ny-(cur_speed-base_speed)

            if ret == 1 :
              sol_moves.append(("move",(cur_x,cur_y)))

        while i < num_turns and hp > 0 :
            i += 1
            hp -= cur_power
            if ret == 1 :
              sol_moves.append(("attack",1,ind))
            if hp <= 0 :
                cur_exp += exp
                cur_g += gold


        L = bisect_right(levels,cur_exp)
        cur_power = int( base_power*(1 + L*(level_power_coeff/100)) )
        cur_speed = int( base_speed*(1 + L*(level_speed_coeff/100)) )
        cur_range= int( base_range*(1 + L*(level_range_coeff/100)) )

    if ret == 1 :
        return sol_moves

    return cur_g

def get_neighbor(sol, n,k,probability ):
    new_sol = sol.copy()
    for _ in range(k) :
      i = random.randint(1,len(sol)-1)
      # if (probability > 0.5) and new_sol[i][5] > new_sol[i-1][5] :
      #   new_sol[i] , new_sol[i-1] = new_sol[i-1] ,new_sol[i]
      # elif (probability < 0.5) and new_sol[i][4] > new_sol[i-1][4] :
      new_sol[i] , new_sol[i-1] = new_sol[i-1] ,new_sol[i]
    return new_sol


def simulated_annealing(n) :
    T = 500
    alpha = 0.9991
    Tmin = 0.9

    cand = monsters.copy()
    initial_eval = objective_function(cand,0)
    best = initial_eval
    ans = cand.copy()
    p = 1

    while T > Tmin :
        new_cand = get_neighbor(cand,n,min(len(monsters)//2,max(1,int(T))),p)
        new_eval = objective_function(new_cand,0)

        delta_E = new_eval - initial_eval

        if delta_E > 0 :
            cand = new_cand
            initial_eval = new_eval
        else :
            p = exp( (delta_E /(T)))
            r = random.random()
            # print((delta_E / T),p,r,t,initial_eval)
            if p > r :
                cand = new_cand
                initial_eval = new_eval

        T *= alpha

        if initial_eval > best :
            ans = cand.copy()
            best = initial_eval

        print(f"current best value: {best}, current temperature : {T}")

    print(best)
    return objective_function(ans,1)

def solve() :
    global moves , posi_atk , posi_move

    posi_move = {}
    max_w = max_h = 1
    min_w = w
    min_h = h

    for k,x,y,hp,exp,gold in monsters:
        max_w = max(max_w,x)
        max_h = max(max_h,y)
        min_w = min(min_w,x)
        min_h = min(min_h,y)


    max_w = max(max_w,hx)
    max_h = max(max_h,hy)
    min_w = min(min_w,hx)
    min_h = min(min_h,hy)
    check = 0
    print(max_w,max_h,min_w,min_h)
    for i in range(min_w,max_w+1) :
        for j in range(min_h,max_h+1) :
            posi_move[i,j] = []
            for x in range(max(min_w,i-base_speed) , min(max_w,i+base_speed) + 1,(base_speed//4)) :
                for y in range(max(min_h,j-base_speed) , min(max_h,j+base_speed) + 1,(base_speed//4)) :
                    if (x,y) == (i,j)  :
                        continue
                    if (x-i)**2 + (y-j)**2 <= base_speed**2  :
                        try :
                            posi_move[i,j].append((x,y))
                        except :
                            posi_move[i,j] = [((x,y))]

    print(min_h,max_h,min_w,max_w,check)
    moves = simulated_annealing(num_turns)
    print(moves)

###############################################################################
###############################################################################
###############################################################################
solve()

Streaming output truncated to the last 5000 lines.
current best value: 2873, current temperature : 80.88984505302746
current best value: 2873, current temperature : 80.81704419247973
current best value: 2873, current temperature : 80.7443088527065
current best value: 2873, current temperature : 80.67163897473905
current best value: 2873, current temperature : 80.59903449966178
current best value: 2873, current temperature : 80.52649536861209
current best value: 2873, current temperature : 80.45402152278034
current best value: 2873, current temperature : 80.38161290340983
current best value: 2873, current temperature : 80.30926945179677
current best value: 2873, current temperature : 80.23699110929014
current best value: 2873, current temperature : 80.16477781729178
current best value: 2873, current temperature : 80.09262951725621
current best value: 2873, current temperature : 80.02054615069068
current best value: 2873, current temperature : 79.94852765915506
current best value: 2873, 

In [28]:
from math import log2

ans = { "moves": [] }
for move in moves:
    if move[0] == "attack":
        for i in range(move[1]):
            if move[0] == "attack":
                ans["moves"].append({
                    "type": "attack",
                    "target_id": int((move[2]))
                })
    else:
        ans["moves"].append({
            "type": "move",
            "target_x": move[1][0],
            "target_y": move[1][1]
        })
print(json.dumps(ans))
f = open("out.txt", "w")
f.write(json.dumps(ans))
submit(test_num, json.dumps(ans))
f.close()

{"moves": [{"type": "move", "target_x": 108, "target_y": 26}, {"type": "move", "target_x": 116, "target_y": 20}, {"type": "move", "target_x": 124, "target_y": 15}, {"type": "attack", "target_id": 15}, {"type": "attack", "target_id": 15}, {"type": "move", "target_x": 116, "target_y": 21}, {"type": "move", "target_x": 108, "target_y": 27}, {"type": "move", "target_x": 100, "target_y": 33}, {"type": "move", "target_x": 92, "target_y": 39}, {"type": "move", "target_x": 84, "target_y": 45}, {"type": "move", "target_x": 76, "target_y": 51}, {"type": "move", "target_x": 68, "target_y": 57}, {"type": "move", "target_x": 60, "target_y": 63}, {"type": "attack", "target_id": 18}, {"type": "attack", "target_id": 18}, {"type": "move", "target_x": 54, "target_y": 55}, {"type": "attack", "target_id": 21}, {"type": "attack", "target_id": 21}, {"type": "move", "target_x": 69, "target_y": 55}, {"type": "move", "target_x": 82, "target_y": 61}, {"type": "move", "target_x": 95, "target_y": 67}, {"type": "a